In [1]:
# Implemented Faiss to make similarity search fast
# Tried VGG16                                                                                     0.592
# Now trying InceptionResNetV2                                                                    0.525
# Applied batching in prediction with Xception                                                    0.558
# EfficientNetB0 , faiss distance threshold reduce 500 -> 25                                      0.497
# EfficientNetB7 , faiss distance threshold reduce 25 -> 10                                       0.389
# EfficientNetB0 , faiss distance threshold reduce 500 -> 1.1e-6  and Normalize images            0.517
# VGG16 , faiss distance threshold = 1  and Normalized images                                     0.601
               
# Arcface efficientB3 threshold = 10  and Normalized images , batch size 32 gpu(0.678)   
# Arcface efficientB3 threshold = 12  and Normalized images , batch size 32 gpu(0.687)             
# Arcface efficientB3 threshold = 8  and Normalized images , batch size 32 gpu(0.666)              
# Arcface efficientB3 threshold = 14  and Normalized images , batch size 32 gpu(0.691)
# Arcface efficientB3 threshold = 16  and Normalized images , batch size 32 gpu(0.686)
# Arcface efficientB3 threshold = 13  and Normalized images , batch size 32 gpu(0.690)
# Arcface efficientB3 threshold = 14.5  and Normalized images , batch size 32 gpu(0.691)
# Arcface efficientB3 threshold = 13.5  and Normalized images , batch size 32 gpu(0.692)
# Arcface efficientB3 threshold = 15  and Normalized images , batch size 32 gpu(0.692)
# Arcface efficientB3 threshold = 15  and Normalized images , batch size 32 gpu, resize_with_pad
# Arcface efficientB3 threshold = 15  and Normalized images , batch size 32 gpu, resize_with_pad, latest model V5
# Arcface efficientB3 threshold = 15  and Normalized images , batch size 32 gpu, resize_with_pad, latest model V9(0.69)
# Arcface efficientB3 threshold = 10  and Normalized images , batch size 32 gpu, resize_with_pad, latest model V9(0.68)
# Arcface efficientB3 threshold = 12.5  and Normalized images , batch size 32 gpu, resize_with_pad, latest model V9(0.69)
# Arcface efficientB3 threshold = 13.75  and Normalized images , batch size 32 gpu, resize_with_pad, latest model V9
# threshold = 13.75  and Normalized images , batch size 32 gpu, resize_with_pad, latest model ragnar715(0.693)
# threshold = 15  and Normalized images , batch size 32 gpu, resize_with_pad, latest model ragnar715

faiss_distance_threshold = 15#20#1#1.1e-6#10#25#500
prediction_batch_size = 32

In [2]:
# !pip install --no-index --find-links '/kaggle/input/faiss-170-latest-cpu-gpu/' faiss-cpu==1.7.0
!pip install --no-index --find-links '/kaggle/input/faiss-170-latest-cpu-gpu/' faiss-gpu==1.7.0
!pip install ../input/arcface-baseline/Keras_Applications-1.0.8-py3-none-any.whl
!pip install /kaggle/input/arcface-baseline/efficientnet-1.1.0-py3-none-any.whl

Looking in links: /kaggle/input/faiss-170-latest-cpu-gpu/
Processing /kaggle/input/faiss-170-latest-cpu-gpu/faiss_gpu-1.7.0-cp37-cp37m-manylinux2014_x86_64.whl
Processing /kaggle/input/arcface-baseline/Keras_Applications-1.0.8-py3-none-any.whl
Processing /kaggle/input/arcface-baseline/efficientnet-1.1.0-py3-none-any.whl


In [3]:
from tqdm import tqdm
from keras.models import load_model, Model
from keras.layers import Input, GlobalAveragePooling2D, Dense
from keras.applications.resnet50 import ResNet50
from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications import InceptionResNetV2, Xception, EfficientNetB0, EfficientNetB7
import efficientnet.tfkeras as efn
from PIL import Image
import tensorflow as tf
import faiss
import os
import cv2
import time
import math
import matplotlib.pyplot as plt
import numpy as np # linear algebra
import pandas as pd # data processing

In [4]:
# model_pwd = '/kaggle/input/siamese-network-with-triplet-loss-base-model/embedding_model.h5'
# embedding_model = load_model(model_pwd)
# IMAGE_SIZE = [224,224]
# image_input=Input(shape=(224,224,3))

# model_pwd ='../input/keras-pretrained-models/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'
# model = VGG16(weights=model_pwd,input_tensor=image_input, include_top=False)

# model_pwd ='../input/keras-pretrained-models/xception_weights_tf_dim_ordering_tf_kernels_notop.h5'
# model = Xception(weights=model_pwd,input_tensor=image_input, include_top=False)

# model_pwd ='../input/efficientnetb0/efficientnetb0_notop.h5'
# model = EfficientNetB0(weights=model_pwd, input_tensor=image_input, include_top=False)

# model_pwd ='../input/efficientnetb7/efficientnetb7_notop.h5'
# model = EfficientNetB7(weights=model_pwd,input_tensor=image_input, include_top=False)
# model = EfficientNetB7(weights='imagenet', input_tensor=image_input, include_top=False)

# x=model.output
# x=GlobalAveragePooling2D()(x)
# x=Dense(512,activation='relu')(x)
# embedding_model = Model(image_input,x)
# embedding_model.summary()

# ArcFace

In [5]:
# Arcmarginproduct class keras layer
class ArcMarginProduct(tf.keras.layers.Layer):
    '''
    Implements large margin arc distance.

    Reference:
        https://arxiv.org/pdf/1801.07698.pdf
        https://github.com/lyakaap/Landmark2019-1st-and-3rd-Place-Solution/
            blob/master/src/modeling/metric_learning.py
    '''
    def __init__(self, n_classes, s=30, m=0.50, easy_margin=False,
                 ls_eps=0.0, **kwargs):

        super(ArcMarginProduct, self).__init__(**kwargs)

        self.n_classes = n_classes
        self.s = s
        self.m = m
        self.ls_eps = ls_eps
        self.easy_margin = easy_margin
        self.cos_m = tf.math.cos(m)
        self.sin_m = tf.math.sin(m)
        self.th = tf.math.cos(math.pi - m)
        self.mm = tf.math.sin(math.pi - m) * m

    def get_config(self):

        config = super().get_config().copy()
        config.update({
            'n_classes': self.n_classes,
            's': self.s,
            'm': self.m,
            'ls_eps': self.ls_eps,
            'easy_margin': self.easy_margin,
        })
        return config

    def build(self, input_shape):
        super(ArcMarginProduct, self).build(input_shape[0])

        self.W = self.add_weight(
            name='W',
            shape=(int(input_shape[0][-1]), self.n_classes),
            initializer='glorot_uniform',
            dtype='float32',
            trainable=True,
            regularizer=None)

    def call(self, inputs):
        X, y = inputs
        y = tf.cast(y, dtype=tf.int32)
        cosine = tf.matmul(
            tf.math.l2_normalize(X, axis=1),
            tf.math.l2_normalize(self.W, axis=0)
        )
        sine = tf.math.sqrt(1.0 - tf.math.pow(cosine, 2))
        phi = cosine * self.cos_m - sine * self.sin_m
        if self.easy_margin:
            phi = tf.where(cosine > 0, phi, cosine)
        else:
            phi = tf.where(cosine > self.th, phi, cosine - self.mm)
        one_hot = tf.cast(
            tf.one_hot(y, depth=self.n_classes),
            dtype=cosine.dtype
        )
        if self.ls_eps > 0:
            one_hot = (1 - self.ls_eps) * one_hot + self.ls_eps / self.n_classes

        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)
        output *= self.s
        return output

In [6]:
# For tf.dataset
AUTO = tf.data.experimental.AUTOTUNE

# Configuration
BATCH_SIZE = 8
IMAGE_SIZE = [512, 512]
# Seed
SEED = 42
# Verbosity
VERBOSE = 1
# Number of classes
N_CLASSES = 11011#11014


margin = ArcMarginProduct(
            n_classes = N_CLASSES, 
            s = 30, 
            m = 0.7, 
            name='head/arc_margin', 
            dtype='float32'
            )

inp = tf.keras.layers.Input(shape = (*IMAGE_SIZE, 3), name = 'inp1')
label = tf.keras.layers.Input(shape = (), name = 'inp2')
x = efn.EfficientNetB3(weights = None, include_top = False)(inp)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = margin([x, label])

output = tf.keras.layers.Softmax(dtype='float32')(x)

model = tf.keras.models.Model(inputs = [inp, label], outputs = [output])
# model.load_weights('../input/efficientnetb3-512-42-v9/EfficientNetB3_512_42_V9.h5')
model.load_weights('../input/ragnar715score/EfficientNetB3_512_42.h5')
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
inp1 (InputLayer)               [(None, 512, 512, 3) 0                                            
__________________________________________________________________________________________________
efficientnet-b3 (Functional)    (None, None, None, 1 10783528    inp1[0][0]                       
__________________________________________________________________________________________________
global_average_pooling2d (Globa (None, 1536)         0           efficientnet-b3[0][0]            
__________________________________________________________________________________________________
inp2 (InputLayer)               [(None,)]            0                                            
______________________________________________________________________________________________

In [7]:
embedding_model = tf.keras.models.Model(inputs = model.input[0], outputs = model.layers[-4].output)
embedding_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inp1 (InputLayer)            [(None, 512, 512, 3)]     0         
_________________________________________________________________
efficientnet-b3 (Functional) (None, None, None, 1536)  10783528  
_________________________________________________________________
global_average_pooling2d (Gl (None, 1536)              0         
Total params: 10,783,528
Trainable params: 10,696,232
Non-trainable params: 87,296
_________________________________________________________________


In [8]:
# from keras.datasets import mnist
# from scipy import spatial
# import math


# (X_train_check, y_train_check), (X_test_check, y_test_check) = mnist.load_data()
# x_0 = embedding_model.predict(X_train_check[0].reshape(-1,28,28,1))
# x_3 = embedding_model.predict(X_train_check[3].reshape(-1,28,28,1))
# x_6 = embedding_model.predict(X_train_check[6].reshape(-1,28,28,1))
# print(x_3)

# # print(spatial.distance.cosine(x_3,x_3))
# # print(spatial.distance.cosine(x_3,x_6))
# # print(spatial.distance.cosine(x_0,x_6))


# # print(cosine_similarity(x_3,x_6))
# # print(cosine_similarity(x_0,x_3))

# Analysing model results

In [9]:
# test_image_path = '/kaggle/input/shopee-product-matching/test_images/'
# image_names = os.listdir(test_image_path)
# image_embeddings = []
# for i in image_names:
#     img = cv2.imread(test_image_path+i, 1)
#     img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
# #     print(img.shape)
# #     plt.imshow(img)
# #     plt.show()
    
#     img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
#     resize_img= cv2.resize(img, (28, 28), interpolation = cv2.INTER_CUBIC)
# #     plt.imshow(resize_img)
# #     plt.show()
#     image_embeddings.append(embedding_model.predict(resize_img.reshape(-1,28,28,1)))
# #     print(embedding_model.predict(resize_img.reshape(-1,28,28,1)))
# #     print('--------------------------')
# print(cosine_similarity(image_embeddings[0],image_embeddings[0]))
# print(cosine_similarity(image_embeddings[0],image_embeddings[1]))
# print(cosine_similarity(image_embeddings[0],image_embeddings[2]))
# print(cosine_similarity(image_embeddings[1],image_embeddings[2]))

# Build Knn/Embedding dictionary and Generating test image embeddings 

In [10]:
# test_csv_path = '/kaggle/input/shopee-product-matching/test.csv'
# # test_csv_path = '/kaggle/input/shopee-product-matching/train.csv'
# test_data = pd.read_csv(test_csv_path)
# test_data.head()

In [11]:
# start_time= time.time()
# test_image_path = '/kaggle/input/shopee-product-matching/test_images/'
# image_names = test_data['image'].values
# posting_ids = test_data['posting_id'].values
# image_embeds = []
# for i,img_name in tqdm(enumerate(image_names)):
#     img = cv2.imread(test_image_path+img_name, 1)
#     img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#     resize_img= cv2.resize(img, (224, 224), interpolation = cv2.INTER_CUBIC)
#     image_embeds.append(embedding_model.predict(resize_img.reshape(-1,224,224,3)))
# # print(time.time()-start_time)
# # del embedding_model

# Batching

In [12]:
# start_time= time.time()
# test_image_path = '/kaggle/input/shopee-product-matching/test_images/'
# # test_image_path = '/kaggle/input/shopee-product-matching/train_images/'
# image_names = test_data['image'].values
# posting_ids = test_data['posting_id'].values
# resize_images = []
# for i,img_name in tqdm(enumerate(image_names)):
#     img = cv2.imread(test_image_path+img_name, 1)
#     img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
# #     print(img.shape)
# #     plt.imshow(img)
# #     plt.show()
    
# #     img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
#     resize_img= cv2.resize(img, (224, 224), interpolation = cv2.INTER_CUBIC)
#     resize_images.append(resize_img)
# #     plt.imshow(resize_img)
# #     plt.show()
# #     print(embedding_model.predict(resize_img.reshape(-1,28,28,1)))
# #     print('--------------------------')
# print(time.time()-start_time)
# print(len(resize_images),np.array(resize_images).shape)
# del image_names

In [13]:
# #Batch implementation

# start_time= time.time()
# image_embeds = []
# batch_size = 2#32
# iteration = math.floor(len(resize_images)/batch_size)
# resize_images = np.array(resize_images)
# for i in tqdm(range(iteration)):
#     image_embeds.append(embedding_model.predict(resize_images[batch_size*i:batch_size*(i+1)]))
# image_embeds.append(embedding_model.predict(resize_images[batch_size*(i+1):]))
# print(time.time()-start_time)
# image_embeds = np.concatenate(image_embeds)
# print(image_embeds.shape)
# del embedding_model
# del resize_images

In [14]:
def prepocess(image):
#     image = tf.image.decode_jpeg(image, channels = 3)
    image = tf.image.resize_with_pad(image, target_width = IMAGE_SIZE[0], target_height = IMAGE_SIZE[1])
#     image = tf.image.resize(image, IMAGE_SIZE)
    image = tf.cast(image, tf.float32) / 255.0
    return image

In [15]:
#Batch implementation
test_csv_path = '/kaggle/input/shopee-product-matching/test.csv'
# test_csv_path = '/kaggle/input/shopee-product-matching/train.csv'
test_data = pd.read_csv(test_csv_path)
test_image_path = '/kaggle/input/shopee-product-matching/test_images/'
# test_image_path = '/kaggle/input/shopee-product-matching/train_images/'
image_names = test_data['image'].values
posting_ids = test_data['posting_id'].values


start_time= time.time()
image_embeds = []
total_images = len(image_names)
if total_images==3:
    batch_size = 2
else:
    batch_size = prediction_batch_size
iteration = math.ceil(total_images/batch_size)
for i in tqdm(range(iteration)):
    resize_images = []
    start = batch_size*i
    end = min(batch_size*(i+1),total_images)
    for img_name in image_names[batch_size*i:batch_size*(i+1)]:
        img = cv2.imread(test_image_path+img_name, 1)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#         resize_img= cv2.resize(img,(IMAGE_SIZE[0], IMAGE_SIZE[1]), interpolation = cv2.INTER_CUBIC)
#         resize_img = resize_img/255.0
        resize_img = prepocess(img)
#         plt.imshow(resize_img)
#         plt.show()
        resize_images.append(resize_img)
    resize_images = np.array(resize_images)
    image_embeds.append(embedding_model.predict(resize_images))
    del resize_images
# image_embeds.append(embedding_model.predict(resize_images[batch_size*(i+1):]))
print(time.time()-start_time)
image_embeds = np.concatenate(image_embeds)
print(image_embeds.shape)


100%|██████████| 2/2 [00:05<00:00,  2.61s/it]

5.2296974658966064
(3, 1536)


In [16]:
del embedding_model
del start_time

# Faiss

In [17]:
d = 1536#512
# xb = np.array(image_embeds).reshape(-1,d)
xb = image_embeds
del image_embeds
index = faiss.IndexFlatL2(d)   # build the index
print(index.is_trained)
index.add(xb)                  # add vectors to the index
print(index.ntotal)

True
3


In [18]:
k = 50                          # we want to see 4 nearest neighbors
# D, I = index.search(xb[:3], k) # sanity check
D, I = index.search(xb, k) # sanity check
# print(I)                       #indexs   
# print(D)                       #Dictances

In [19]:
del xb
del index

In [20]:
matched_list = []
for i,vali in enumerate(I):
    temp_list = ''
    for j,valj in enumerate(vali):
        if valj==-1:
            break
        if D[i][j]>faiss_distance_threshold:
            break
        temp_list+=posting_ids[valj]+' '
    matched_list.append(temp_list)

In [21]:
del D
del I

# Preparing submission file

In [22]:
df = pd.DataFrame({
    'posting_id':posting_ids, 
    'matches':matched_list
})
del posting_ids
del matched_list

In [23]:
df.to_csv('submission.csv',index=False)

In [24]:
df.head()

,posting_id,matches
0,test_2255846744,test_2255846744
1,test_3588702337,test_3588702337
2,test_4015706929,test_4015706929
